In [ ]:
# Import comet_ml at the top of your file
# from comet_ml import Experiment

from matplotlib import pyplot as plt
# from matplotlib import style
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import swifter
from sklearn import metrics
import itertools
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras as k
import tensorflow as tf
# import torch
from scipy import stats
import nltk


# Create an experiment with your api key
# experiment = Experiment(
#     api_key="BKac2uRt0FMAlheXf6HClaZhD",
#     project_name="general",
#     workspace="sarthak7034",
# )


In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# torch.cuda.is_available()

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english') 

In [ ]:
# Function to plot the confusion matrix (code from https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html)


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# experiment.end()

In [ ]:
#Read dataset
data = pd.read_csv("./all_data.csv").drop("Unnamed: 0", axis=1)

In [ ]:
def clean_text(text):
    #Remove URLs
    text = re.sub(r"http\S+", "", text)
    #Tokenize
    tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
    tokens = tokenizer.tokenize(text)
    # remove stopwords
    tokens = [w for w in tokens if not w in stop_words] 
    #Remove non alphanumerica characters
    words = [word for word in tokens if word.isalpha()]    
    return words

scaler = MinMaxScaler()

In [ ]:
#Applying lamdba function to clean text
data['text'] = data.swifter.apply(lambda row: clean_text(row['text']), axis=1)
#Clean title
data['title'] = data.swifter.apply(lambda row: clean_text(row['title']), axis=1)

#Replacing columns with cleaned ones
data = data[data['title'].str.len() >= 1]
data = data[data['text'].str.len() >= 1]


In [ ]:
df1 = data.loc[:,['anger','anticipation','disgust','fear','joy','sadness','surprise','trust','negative','positive']]

# Zero Score Dataframe Normalization
MaxMin_df = stats.zscore(df1)
MaxMin_df.head()

In [ ]:
#Displaying the new dataset which we will be working with
df2 = data.loc[:,['title','text','type']]
df2

In [ ]:
# Whether any values are missing
# df2.isnull().sum()
# df1.isnull().sum()

In [ ]:
# Merging columns
# df2['content'] = df2['title'] + df2['text']
# df2

In [ ]:
# How many fake and real articles?
print(df2.groupby(['type'])['title'].count())
df2.groupby(['type'])['title'].count().plot(kind="bar")
plt.show()

In [ ]:
df2 = df2[df2['text'].map(len) >= 1]
#Reset index
df2 = df2.reset_index().drop("index", axis=1)
df2


In [ ]:
model = KeyedVectors.load_word2vec_format("./GoogleNews-vectors-negative300.bin.gz",binary=True)

In [ ]:
# gg = np.zeros((50000,300))
# for i, e in model.key_to_index.items():
#     gg[model.get_index('e')] = model[i]
print(model.key_to_index.items())

In [ ]:
# Embedding Matrix
embed_mat = np.zeros((50000,300))
for i, e in model.key_to_index.items():
    # embed_mat[model.key_to_index['e']] = model[i]
    embed_mat[model.get_index('e')] = model[i]


In [ ]:
# Applying it to the columns (Topic modeling and document similarity analysis)

df2['title'] = df2.apply(lambda r: [model.key_to_index[x] for x in r['title'] if x in model.key_to_index], axis=1)
df2['text'] = df2.apply(lambda r: [model.key_to_index[x] for x in r['text'] if x in model.key_to_index], axis=1)
# df2['content'] = df2.apply(lambda r: [model.key_to_index[x] for x in r['content'] if x in model.key_to_index], axis=1)


In [ ]:
# One-hot encoding
encoder = LabelBinarizer().fit(list(df2['type']))
df2['type'] = df2.swifter.apply(lambda r: encoder.transform([r['type']])[0], axis=1)
df2

In [ ]:
def pad_array(array, token_len):
    diff_token = token_len - len(array)
    if(diff_token < 0):
        array = array[:token_len] #Truncate
    else:
        #Pad
        array += [0]*diff_token 
        
    return array 

# tokens_numbers_text = df2.swifter.apply(lambda row: len(row['text']), axis = 1)
# max_tokens_text = int(np.mean(tokens_numbers_text) + 2 * np.std(tokens_numbers_text))

# tokens_numbers_title = df2.swifter.apply(lambda row: len(row['title']), axis = 1)
# max_tokens_title = int(np.mean(tokens_numbers_title) + 2 * np.std(tokens_numbers_title))
# print(max_tokens_text,max_tokens_title)    

# Check the highest length of text and title:
max_tokens_text = 10000
max_tokens_title = 48


In [ ]:
df2['text'] = df2.swifter.apply(lambda r: pad_array(r['text'], max_tokens_text) , axis=1)
df2['title'] = df2.swifter.apply(lambda r: pad_array(r['title'], max_tokens_title) , axis=1)

# df2['text'] = pad_sequences(df2['text'], maxlen=max_tokens_text,padding='post')
# df2['title'] = pad_sequences(df2['title'], maxlen=max_tokens_title,padding='post')

In [ ]:
#Train - test
X_train, X_test, y_train, y_test = train_test_split(
    df2[['text', 'title']], df2['type'], test_size=0.2, random_state=42)

#Train - valid
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42)
 

train_fit = [np.asarray(X_train['title'].tolist()), 
             np.asarray(X_train['text'].tolist())]

valid_fit = [np.asarray(X_valid['title'].tolist()), 
             np.asarray(X_valid['text'].tolist())]

test_fit = [np.asarray(X_test['title'].tolist()), 
             np.asarray(X_test['text']  .tolist())]    


In [ ]:
#input_title
title_input = k.layers.Input(shape=(max_tokens_title,), name='title_input')
inp = k.layers.Embedding(output_dim=300, input_dim=50000, weights=[embed_mat], trainable=False, name='embed_title')(title_input)

#Added
x = k.layers.Conv1D(filters = 5, kernel_size=4, strides=2, activation='relu', name='Conv1Title')(inp)
x = k.layers.MaxPool1D(pool_size = 2, name='Pool1Title')(x)
x = k.layers.Flatten()(x)
x = k.layers.Dense(50, activation='relu', name='Dense1Title', kernel_regularizer='l2')(x)

#input_Title
text_input = k.layers.Input(shape=(max_tokens_text,), name='text_input')
inp2 = k.layers.Embedding(output_dim=300, input_dim=50000, weights=[embed_mat], trainable=False, name='embed_text')(text_input)
x2 = k.layers.Conv1D(filters = 40, kernel_size = 16, strides = 2, activation='relu', name='Conv1Text')(inp2)
x2 = k.layers.MaxPool1D(pool_size = 4, name='Pool1Text')(x2)

#Added
x2 = k.layers.Conv1D(filters = 80, kernel_size = 16, strides = 4, activation='relu', name='Conv2Text')(inp2)
x2 = k.layers.MaxPool1D(pool_size = 4, name='Pool2Text')(x2)
x2 = k.layers.Flatten()(x2)
x2 = k.layers.Dense(100, activation='relu', kernel_regularizer='l2', name='Dense1Text')(x2)


#Merge
x = k.layers.concatenate([x, x2])

#Common part
x = k.layers.Dense(50, activation='relu')(x)
x = k.layers.Dropout(0.2)(x)
x = k.layers.Dense(50, activation='relu')(x)
x = k.layers.Dropout(0.2)(x)
out = k.layers.Dense(1, activation='sigmoid')(x)


#Build model
model = k.models.Model(inputs=[title_input, text_input], outputs=[out])
model.compile(k.optimizers.RMSprop(), loss='binary_crossentropy', metrics=['acc'])

model.summary()

In [ ]:
hist = model.fit(x=train_fit, y=np.asarray(y_train.tolist()), batch_size=128, epochs=20,
        callbacks = [k.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta = 0.1)], validation_data=(valid_fit, np.array(y_valid.tolist())))

In [ ]:
# # Confusion Matrix of SVM
# cm = metrics.confusion_matrix(y_test, prediction)
# plot_confusion_matrix(cm, classes=['Fake', 'Real'])